# Tests Loss
On this notebook we will exercise the expected input/target for segmentation loss.
```python
n_class = 10
batch_size = 4
img_size = 224
preds = torch.randn(batch_size, num_classes, img_size, img_size)
labels = torch.empty(batch_size, img_size, img_size, dtype=torch.long).random_(n_class)

criterion = nn.CrossEntropyLoss()
loss = criterion(preds, labels)
```
#### References
* https://discuss.pytorch.org/t/the-cost-function-for-semantic-segmentation/27668/2
* https://discuss.pytorch.org/t/multi-class-cross-entropy-loss-function-implementation-in-pytorch/19077/12
* https://github.com/CSAILVision/semantic-segmentation-pytorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import models
print("PyTorch Version: ",torch.__version__)
num_classes = 3
batch_size = 100
img_size = 224

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
print('Device:', device)
num_gpu = torch.cuda.device_count()
batch_size *= num_gpu
print('Number of GPUs Available:', num_gpu)

PyTorch Version:  1.1.0
Device: cuda:0
Number of GPUs Available: 8


#### Define expected labels, model output

In [2]:
preds = torch.randn(batch_size, num_classes, img_size, img_size).to(device)
labels = torch.empty(batch_size, img_size, img_size, dtype=torch.long).random_(num_classes).to(device)
print('preds.shape:', preds.shape)
print('labels.shape:', labels.shape)

preds.shape: torch.Size([800, 3, 224, 224])
labels.shape: torch.Size([800, 224, 224])


#### Try Cross-Entropy Loss
If you use this loss you don't need to use the softmax activation

In [3]:
criterion = nn.CrossEntropyLoss()
loss = criterion(preds, labels)
print('Loss:',loss)

Loss: tensor(1.3899, device='cuda:0')


#### Try on Model

In [4]:
img_reference = torch.randn(batch_size,3,224,224).to(device)
img_test = torch.randn(batch_size,3,224,224).to(device)
change_net = models.ChangeNet(num_classes=num_classes)
if num_gpu > 1:
    change_net = nn.DataParallel(change_net)
change_net = change_net.to(device)

preds = change_net([img_reference, img_test])
print('preds.shape:', preds.shape)
criterion = nn.CrossEntropyLoss()
loss = criterion(preds, labels)
print('Loss:',loss)

preds.shape: torch.Size([800, 3, 224, 224])
Loss: tensor(1.3680, device='cuda:0', grad_fn=<NllLoss2DBackward>)
